In [1]:
#Standard Imports
import os
import sys
import pandas as pd
import numpy as np
# from collections import defaultdict
import timeit
import argparse
import logging
import copy
import numpy.random as rnd

sys.path.append('../')

# Custom Imports
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput, Harmony
from src.midi_processing import *
# from src.learning_weights import *

import src.evaluate
import src.music_functions
import src.ga_model

In [2]:
# Importing Musical Corpus
#musical_work_df = pd.read_csv("../data/test_melody_hatikvah(israel)_4_1_minor_1.mid")
#musical_corpus = []
#for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
#    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [3]:
input_midi = "../data/test_melody_hatikvah(israel)_4_1_minor_1.mid"
#melody, tempo_interval, meter, key, tonality, first_on_beat = midi_to_array(input_midi)
melody, tempo_interval = midi_to_array_quick(input_midi)
filename = str.split(str.split(input_midi,'/')[-1],'_')
first_on_beat = int(str.split(filename[-1],'.')[0])
tonality = filename[-2]
key = int(filename[-3])
meter = int(filename[-4])
song_title = '_'.join(filename[2:-4])
musical_corpus = []
musical_corpus.append(MusicalWorkInput(song_title, meter, key, tonality, first_on_beat, list(filter(None,melody[0]))))

In [4]:
musical_work_df = pd.read_csv('../data/sample_input.csv')
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [5]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [6]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [7]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [8]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [9]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [10]:
# Model
for musical_work in [musical_corpus[-5]]:

    print('*'*20)
    print('Work: {}'.format(musical_work.title))
    print('*'*20)
    
    if musical_work.tonality == 'major':
        penalties_chord_progression = penalties_chord_progression_major
        chord_vocab = chord_vocab_major
    else:
        penalties_chord_progression = penalties_chord_progression_minor
        chord_vocab = chord_vocab_minor        
    
    #Solving Model
    ga_model = src.ga_model.GAmodel(musical_work, chord_vocab, max_generation=100, population_size=100,
                                    hard_constraints=hard_constraints,
                                    soft_constraint_w_weights=soft_constraint_w_weights,
                                    chord_progression_penalties=penalties_chord_progression,
                                    mutation_probability=[0.6, 0.9])
    solution, midi_array, progress_array = ga_model.solve()

********************
Work: Ach Gott, erhor', mein Seufzen und Wehklagen (8)
********************
[[33. 33. 33. 31. 29. 28. 28. 26.]
 [21. 31. 28. 27. 21. 31. 22. 21.]
 [21. 22. 19. 19. 21. 16. 19. 18.]
 [18. 22. 19.  7. 21. 10.  7. 18.]]
1657.3125
****************************************************************************************************
[[33. 33. 33. 31. 29. 28. 28. 26.]
 [30. 28. 33. 29. 19. 21. 28. 30.]
 [26. 21. 25. 17. 28. 16. 25. 18.]
 [26. 24. 16. 17. 19. 16. 25.  6.]]
1660.5
****************************************************************************************************
[[33. 33. 33. 31. 29. 28. 28. 26.]
 [26. 33. 24. 29. 19. 33. 33. 26.]
 [18. 21. 21. 17. 12. 21. 21. 14.]
 [26.  6.  5.  9. 16.  9. 17.  9.]]
1905.3125
****************************************************************************************************
[[33. 33. 33. 31. 29. 28. 28. 26.]
 [21. 29. 29. 19. 21. 35. 31. 21.]
 [18. 12. 22. 28. 14. 23. 16. 18.]
 [18. 21. 22. 19.  9. 20. 12. 26.]]
1906.125


In [11]:
for x in midi_array:
    print(type(x[0]))

<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>


In [12]:
print(solution, type(solution))

    0   1   2   3   4   5   6   7
0  33  33  33  31  29  28  28  26
1  30  28  25  27  21  31  22  21
2  26  21  19  19  21  16  19  18
3  26  13  16   7  21  10   7  18
4   1   6  11   2   5   3   3   1 <class 'pandas.core.frame.DataFrame'>
